# Discover EOPF Zarr - Sentinel-2 L2A

## Introduction
This tutorial introduces you to the structure of a Zarr sample for **Sentinel-2 L2A** data. We will demonstrate how to access and open a Zarr product sample with `xarray`, how to visualise the `zarr` encoding structure, explore embedded information, and retrieve relevant metadata for further processing.

### Prerequisites
This tutorial uses a re-processed sample dataset frome the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/) that is available for download [here](https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr). 

The selected `zarr` product is a Sentinel-2 L2A tile from the 10th of June 2025 (file name: `S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr.`).

<hr>

#### Import libraries

In [1]:
import os
import xarray as xr

#### Helper functions

##### `print_gen_structure`
This function helps us to retrieve an visualise the names for each of the stored groups inside a `zarr`. As an output, it will print a general overview of elements inside the `zarr`.

In [2]:
def print_gen_structure(node, indent=""):
    print(f"{indent}{node.name}")     #allows us access each node
    for child_name, child_node in node.children.items(): #loops inside the selected nodes to extract naming
        print_gen_structure(child_node, indent + "  ") # prints the name of the selected nodes

<hr>

## Open a Zarr Store

In a first step, we use the function `open_datatree()` from the `xarray` library to open a Zarr store as a DataTree.<br>
Inside, we ned to define the following key word arguments:<br>

- `filename_or_obj`: path leading to a `zarr` store
- `engine`: `eopf-zarr`, designed for the EOPF `zarr` by ESA. 
- `op_mode`: extension by the `xarray-eopf` development for allowing an analysis or native mode. For more information visit the [xarray-eopf](https://eopf-sample-service.github.io/xarray-eopf/) documentation.
- `chunks`: loads the data with dask using the engine’s preferred chunk size, generally identical to the format’s chunk size

**Note**: The final print of the `DataTree` object is commented out, as the display can be quite verbose, showing the entire content within the Zarr. An alternative is to apply a helper function that only displays the higher level structure as shown in the next code cell.

In [ ]:
s2l2a_zarr_sample= xr.open_datatree('https://objects.eodc.eu/e05ab01a9d56408d82ac32d69a5aae2a:202506-s02msil2a/10/products/cpm_v256/S2C_MSIL2A_20250610T103641_N0511_R008_T32UMD_20250610T132001.zarr',
    engine="eopf-zarr", # storage format
    op_mode="native", # no analysis mode
    chunks={}, # allows to open the default chunking
)

If we apply the helper function `print_gen_structure` on the root of the DataTree object, we will get a listing of the tree-like structure of the object. We can see all Zarr groups, such as `measurements`, `quality` and `conditions`, their sub-groups and content.

In [6]:
print("Zarr Sentinel 2 L2A Structure")
print_gen_structure(s2l2a_zarr_sample.root) 
print("-" * 30)

Zarr Sentinel 2 L2A Structure
None
  conditions
    geometry
    mask
      detector_footprint
        r10m
        r20m
        r60m
      l1c_classification
        r60m
      l2a_classification
        r20m
        r60m
    meteorology
      cams
      ecmwf
  measurements
    reflectance
      r10m
      r20m
      r60m
  quality
    atmosphere
      r10m
      r20m
      r60m
    l2a_quicklook
      r10m
      r20m
      r60m
    mask
      r10m
      r20m
      r60m
    probability
      r20m
------------------------------


## Extract information from Zarr groups

In a next step, we can explore the content of individual Zarr groups. By specifying the name of the group and subgroup and adding it into square brackets, we can extract the content of the relevant group. Let us for example extract the content of the subgroup `reflectance` under `measurements`.

As a result, it is visible that there are three subgroups of the parent node `measurements/reflectance`: `r10`, `r20` and `r60`, which are the DataArrays with the three different resolutions of the Sentinel-2 L2A data.

The `xarray.DataTree` structure allows the exploration of additional group-related metadata and information. For example, we can find the `chunksize` of each array and the coordinates.

In [7]:
# Retrieving the reflectance groups:
s2l2a_zarr_sample["measurements/reflectance"]

<xarray.DataTree 'reflectance'>
Group: /measurements/reflectance
├── Group: /measurements/reflectance/r10m
│       Dimensions:  (y: 10980, x: 10980)
│       Coordinates:
│         * x        (x) int64 88kB 399965 399975 399985 399995 ... 509735 509745 509755
│         * y        (y) int64 88kB 5900035 5900025 5900015 ... 5790265 5790255 5790245
│       Data variables:
│           b02      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b03      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b04      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│           b08      (y, x) float64 964MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
├── Group: /measurements/reflectance/r20m
│       Dimensions:  (y: 5490, x: 5490)
│       Coordinates:
│         * x        (x) int64 44kB 399970 399990 400010 400030 ... 509710 509730 509750
│         * y        (y) int64 44kB 5900030 5900010 5899990 ... 5790290 5790270 5790250
│       Data variables:
│           b01      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b02      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b03      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b04      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b05      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b06      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b07      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b11      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b12      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
│           b8a      (y, x) float64 241MB dask.array<chunksize=(915, 915), meta=np.ndarray>
└── Group: /measurements/reflectance/r60m
        Dimensions:  (y: 1830, x: 1830)
        Coordinates:
          * x        (x) int64 15kB 399990 400050 400110 400170 ... 509610 509670 509730
          * y        (y) int64 15kB 5900010 5899950 5899890 ... 5790390 5790330 5790270
        Data variables:
            b01      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b02      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b03      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b04      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b05      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b06      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b07      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b09      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b11      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b12      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>
            b8a      (y, x) float64 27MB dask.array<chunksize=(305, 305), meta=np.ndarray>

## Extract Zarr metadata on different levels

Through `s2l2a_zarr_sample.attrs[]` we are able to visualise both the `stac_discovery` and `other_metadata`. <br>
<br>
For the properties inside `stac_discovery` for example:

In [8]:
# STAC metadata style:
s2l2a_zarr_sample.attrs["stac_discovery"]['properties']

{'bands': [{'center_wavelength': 444.2,
   'common_name': 'coastal',
   'full_width_half_max': 0.02,
   'name': 'b01',
   'solar_illumination': 1914.61},
  {'center_wavelength': 489.0,
   'common_name': 'blue',
   'full_width_half_max': 0.065,
   'name': 'b02',
   'solar_illumination': 1972.03},
  {'center_wavelength': 560.6,
   'common_name': 'green',
   'full_width_half_max': 0.035,
   'name': 'b03',
   'solar_illumination': 1822.48},
  {'center_wavelength': 666.5,
   'common_name': 'red',
   'full_width_half_max': 0.03,
   'name': 'b04',
   'solar_illumination': 1506.74},
  {'center_wavelength': 707.1,
   'common_name': 'rededge',
   'full_width_half_max': 0.015,
   'name': 'b05',
   'solar_illumination': 1410.66},
  {'center_wavelength': 741.1,
   'common_name': 'rededge',
   'full_width_half_max': 0.015,
   'name': 'b06',
   'solar_illumination': 1287.62},
  {'center_wavelength': 784.7,
   'common_name': 'rededge',
   'full_width_half_max': 0.02,
   'name': 'b07',
   'solar_illumi

And from `other_metadata`, we are able to retrieve the information specific to a band, for example the **red** reflectance band `b04`:

In [9]:
# Complementing metadata:
s2l2a_zarr_sample.attrs["other_metadata"]['band_description']['b04']

{'bandwidth': 30.0,
 'central_wavelength': 666.5,
 'onboard_compression_rate': '2.97',
 'onboard_integration_time': '1.416499',
 'physical_gain': '6.52683435',
 'spectral_response_step': '1',
 'spectral_response_values': '0.00072495 0.00067953 0.00055633 0.00044453 0.00038798 0.00036298 0.00036151 0.0003695 0.00039185 0.00043727 0.00049592 0.00059393 0.00076071 0.00104977 0.00157623 0.00248616 0.00447622 0.00954832 0.02403303 0.06759162 0.17739783 0.41429503 0.73522461 0.92808255 0.96462147 0.96480885 0.96204362 0.95539632 0.94765049 0.94241519 0.93945425 0.93887659 0.94088614 0.9446717 0.95148654 0.95881145 0.96570839 0.97126869 0.97615516 0.98197906 0.98850481 0.99518579 0.99956988 1 0.99759436 0.99338082 0.98592818 0.97236683 0.9529575 0.92344847 0.85543935 0.68821494 0.4182541 0.18824224 0.07465708 0.03282836 0.01528275 0.00761471 0.00415471 0.00250253 0.00167378 0.00117426 0.00085817 0.00066068 0.00053252 0.00045262 0.00039405 0.00034974 0.00031701 0.00029853 0.0002817 0.00027444 

## Now it is your turn

As we are able to retrieve several items from the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/), lets try the following:
<br>
1. Go to the [Sentinel-2 Level-2A collection](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a)
2. Choose an item of interest.
3. Replicate the workflow and explore the item's `datetime`. When was it retrieved?
4. What are the dimensions?
5. What is the detailed location of the item?

## Conclusion
This tutorial provides an initial understanding of the `zarr` structure for a Sentinel-2 L2A sample.

By using the `xarray` library, we can effectively navigate and inspect the different components within the `zarr` format, including its metadata and array organisation. 
This foundation will help deeply undestand the subsequent data analysis and processing workflows intended in our EOPF 101 series.